In [ ]:
# conda install -c conda-forge geopandas

In [ ]:
# pip install geopy

In [11]:
import pandas as pd
import re

# https://www.geeksforgeeks.org/how-to-get-geolocation-in-python/
# https://www.geeksforgeeks.org/find-the-location-with-specified-latitude-and-longitude-using-python/
from geopy.geocoders import Nominatim


In [2]:
pd.options.display.max_columns = 30
pd.options.display.max_rows = 300

In [3]:
df = pd.read_csv("Cyclistic_202010_202109.csv")
df.head()

D:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13


In [4]:
df.isnull().sum()

Unnamed: 0                 0
ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    523467
start_station_id      523781
end_station_name      567268
end_station_id        567501
start_lat                  0
start_lng                  0
end_lat                 4821
end_lng                 4821
member_casual              0
riding_days                0
riding_time                0
dtype: int64

In [5]:
start_station = df.start_station_name.value_counts()
start_station_list = start_station.index.to_list()
len(start_station_list)

784

In [346]:
(df['start_lat_short']+', '+df['start_lng_short']).head()

0    41.926, -87.638
1    41.948, -87.663
2    41.773, -87.585
3    41.950, -87.659
4    41.928, -87.663
dtype: object

In [6]:
df['start_lat_short'] = df.start_lat.astype(str).str[:6]
df['start_lng_short'] = df.start_lng.astype(str).str[:7]
df['lat_lng_short'] = df['start_lat_short']+', '+df['start_lng_short']
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"


In [152]:
# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")

locations are a bit different, but similar

In [211]:
# a['start_lat_short'].duplicated().sum()
df1 = df[:]
# df1['start_lat_short'].duplicated().sum()
df1 = df1.drop_duplicates(subset=['start_lat_short'])
df1 = df1.reset_index().drop(columns = ['index', 'Unnamed: 0'])

In [347]:
df1.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng,full_address,country,zip_code
0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926100,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926100, -87.638977","Lakeview Ave & Fullerton Pkwy, North Lakeview ...",United States,60614
1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948172, -87.663911","CVS Pharmacy, 3637, North Southport Avenue, Wr...",United States,60613
2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585371,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773462, -87.585371","Stony Island Ave & 67th St, East 67th Street, ...",United States,60619
3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950854,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950854, -87.659244","Clark St & Grace St, West Grace Street, Pine G...",United States,60613
4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928856,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928856, -87.663962","Southport Ave & Wrightwood Ave, West Wrightwoo...",United States,60614


In [213]:
df1['start_lat'] = df1['start_lat'].astype(str).str[:9]
df1['start_lng'] = df1['start_lng'].astype(str).str[:10]
df1['lat_lng'] = df1['start_lat']+", "+df1['start_lng']
df1.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng
0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926100,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926100, -87.638977"
1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948172, -87.663911"
2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585371,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773462, -87.585371"
3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950854,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950854, -87.659244"
4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928856,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928856, -87.663962"


In [197]:
df = df.rename(columns = {'Unnamed: 0' : 'no'})
df.head()

,no,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"


In [184]:
df['lat_lng_short'].nunique()

2405

In [203]:
len(df)

5136261

In [202]:
df.duplicated('lat_lng_short').sum()

5133856

In [204]:
5136261 - 5133856

2405

In [217]:
df1 = df[:]

df1 = df1.drop_duplicates('lat_lng_short')
print(len(df1))
df1.head(100)

2405


,no,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"
5,5,DA6C3759660133DA,electric_bike,2020-10-29 17:38:04,2020-10-29 17:45:43,Larrabee St & Division St,359.0,Wells St & Huron St,53.0,41.903532,-87.643345,41.894397,-87.634312,casual,0,00:07:39,41.903,-87.643,"41.903, -87.643"
6,6,C2F3808FD56B4F84,electric_bike,2020-10-29 09:03:06,2020-10-29 09:17:56,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.925841,-87.639042,41.890470,-87.625896,casual,0,00:14:50,41.925,-87.639,"41.925, -87.639"
7,7,15B13B5A508BA2B6,electric_bike,2020-10-29 16:37:21,2020-10-29 16:52:40,Rush St & Hubbard St,125.0,Lakeview Ave & Fullerton Pkwy,313.0,41.890310,-87.626259,41.925743,-87.638908,casual,0,00:15:19,41.890,-87.626,"41.890, -87.626"
8,8,285D224410C101C5,electric_bike,2020-10-28 23:12:03,2020-10-28 23:24:32,NaN,NaN,Wabash Ave & Grand Ave,199.0,41.900000,-87.670000,41.891222,-87.627432,casual,0,00:12:29,41.9,-87.67,"41.9, -87.67"
9,9,E1FB79FFE6DB0117,electric_bike,2020-10-29 16:38:44,2020-10-29 16:50:17,Canal St & Madison St,174.0,Fairbanks St & Superior St,635.0,41.882372,-87.639551,41.895310,-87.620319,casual,0,00:11:33,41.882,-87.639,"41.882, -87.639"


In [222]:
df1 = df1.reset_index()
df1 = df1.drop(columns = {'no'})
df1 = df1.drop(columns = {'level_0'})
df1

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,5094558,7C5C4BFBB56779F6,electric_bike,2021-09-10 20:58:41,2021-09-10 21:13:18,Paulina St & 18th St,TA1307000159,NaN,NaN,41.858049,-87.669063,41.840000,-87.720000,member,0,00:14:37,41.858,-87.669,"41.858, -87.669"
2401,5098438,A3DCC3B4E8387957,electric_bike,2021-09-08 08:50:54,2021-09-08 09:42:08,NaN,NaN,NaN,NaN,41.650000,-87.560000,41.570000,-87.540000,casual,0,00:51:14,41.65,-87.56,"41.65, -87.56"
2402,5109326,8E34E64D48083921,electric_bike,2021-09-17 11:52:47,2021-09-17 12:30:34,NaN,NaN,NaN,NaN,41.830000,-87.710000,41.920000,-87.700000,member,0,00:37:47,41.83,-87.71,"41.83, -87.71"
2403,5110369,5742BE0463806D7F,electric_bike,2021-09-29 15:14:31,2021-09-29 15:35:13,Noble St & Milwaukee Ave,13290,NaN,NaN,41.899018,-87.662521,41.870000,-87.710000,casual,0,00:20:42,41.899,-87.662,"41.899, -87.662"


In [226]:
# Check if they have the same values 
a = set(df_lat_lng['lat_lng_short'])
b = set(df1['lat_lng_short'])

print("Additional values in first list:", (set(a).difference(b)))
print("Additional values in first list:", (set(b).difference(a)))


Additional values in first list: set()
Additional values in first list: set()


In [200]:
df_lat_lng = pd.DataFrame(df.groupby(['lat_lng_short'])['lat_lng_short'].count())
df_lat_lng = df_lat_lng.rename(columns = {'lat_lng_short' : 'count'}).reset_index()
df_lat_lng

,lat_lng_short,count
0,"41.64, -87.54",3
1,"41.64, -87.62",2
2,"41.64, -87.69",1
3,"41.648, -87.546",73
4,"41.65, -87.53",8
...,...,...
2400,"42.07, -87.71",4
2401,"42.07, -87.72",27
2402,"42.07, -87.73",12
2403,"42.08, -87.68",1


## skip

Separate df_lat_lng into three as geo_loc.geocode() has a access time limit

In [292]:
df1['start_lat_lng'] = df1['start_lat'].astype(str).str[:9]+', '+df1['start_lng'].astype(str).str[:10]
df1['end_lat_lng'] = df1['end_lat'].astype(str).str[:9]+', '+df1['end_lng'].astype(str).str[:10]
df1.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,full_address,start_lat_lng,end_lat_lng
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638","325, West Fullerton Parkway, Mid-North Distric...","41.926100, -87.638977","41.890345, -87.626068"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663","3630, North Wayne Avenue, Wrigleyville, Lake V...","41.948172, -87.663911","41.929527, -87.707817"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585","6717, South Cornell Avenue, Parkside, South Sh...","41.773462, -87.585371","41.791454, -87.600049"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659","3740-3742, North Clark Street, Wrigleyville, L...","41.950854, -87.659244","41.952809, -87.650103"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663","2533, North Southport Avenue, Lincoln Park, Ch...","41.928856, -87.663962","41.917776, -87.691433"


In [293]:
df_lat_lng1 = df1.loc[:802]
df_lat_lng2 = df1.loc[803:1605]
df_lat_lng3 = df1.loc[1606:]

print(len(df_lat_lng1),
      len(df_lat_lng2),
      len(df_lat_lng3)
     )

803 803 799


In [240]:
# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")

loc_list1 = []
loc_list2 = []
loc_list3 = []

# As df_lat_lng1 start from 0, I don't need to set the range specifically
for i in range(len(df_lat_lng1)):
    zip_code = geo_loc.geocode(df_lat_lng1['lat_lng_short'][i]).address
    loc_list1.append(zip_code)
print('df_lat_lng1 - Done')

# Set the range as df_lat_lng2 starts from 802 and ends at 1604
for j in range(df_lat_lng2['lat_lng_short'].index.start, df_lat_lng2['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng2['lat_lng_short'][j]).address
    loc_list2.append(zip_code)
print('df_lat_lng2 - Done')

# Set the range as df_lat_lng3 starts from 1605 and ends at 2404
for k in range(df_lat_lng3['lat_lng_short'].index.start, df_lat_lng3['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng3['lat_lng_short'][k]).address
    loc_list3.append(zip_code)
print('df_lat_lng3 - Done')


df_lat_lng1 - Done
df_lat_lng2 - Done
df_lat_lng3 - Done


Merge loc_list1, 2 and 3 into loc_list and add it to df_lat_lng['full_address']. 

I saved this df because to avoid the above time-consuming process.

In [261]:
loc_list = loc_list1 + loc_list2 + loc_list3
df1['full_address'] = loc_list
df1.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,full_address
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638","325, West Fullerton Parkway, Mid-North Distric..."
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663","3630, North Wayne Avenue, Wrigleyville, Lake V..."
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585","6717, South Cornell Avenue, Parkside, South Sh..."
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659","3740-3742, North Clark Street, Wrigleyville, L..."
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663","2533, North Southport Avenue, Lincoln Park, Ch..."


## to here

In [597]:
df1.to_csv('df_lat_lng.csv')
df_lat_lng = pd.read_csv('df_lat_lng.csv')
df_lat_lng = df_lat_lng.drop(columns = {'Unnamed: 0'})
df_lat_lng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               2405 non-null   int64  
 1   ride_id             2405 non-null   object 
 2   rideable_type       2405 non-null   object 
 3   started_at          2405 non-null   object 
 4   ended_at            2405 non-null   object 
 5   start_station_name  1742 non-null   object 
 6   start_station_id    1742 non-null   object 
 7   end_station_name    1564 non-null   object 
 8   end_station_id      1562 non-null   object 
 9   start_lat           2405 non-null   float64
 10  start_lng           2405 non-null   float64
 11  end_lat             2405 non-null   float64
 12  end_lng             2405 non-null   float64
 13  member_casual       2405 non-null   object 
 14  riding_days         2405 non-null   int64  
 15  riding_time         2405 non-null   object 
 16  start_

In [598]:
df_lat_lng['zip_code'] = df_lat_lng['full_address'].str[-20:-15]
df_lat_lng.head()

,index,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,full_address,start_lat_lng,end_lat_lng,address,zip_code
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638","325, West Fullerton Parkway, Mid-North Distric...","41.926100, -87.638977","41.890345, -87.626068","Lakeview Ave & Fullerton Pkwy, North Lakeview ...",60614
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663","3630, North Wayne Avenue, Wrigleyville, Lake V...","41.948172, -87.663911","41.929527, -87.707817","CVS Pharmacy, 3637, North Southport Avenue, Wr...",60613
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585","6717, South Cornell Avenue, Parkside, South Sh...","41.773462, -87.585371","41.791454, -87.600049","Stony Island Ave & 67th St, East 67th Street, ...",60649
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659","3740-3742, North Clark Street, Wrigleyville, L...","41.950854, -87.659244","41.952809, -87.650103","Clark St & Grace St, West Grace Street, Pine G...",60613
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663","2533, North Southport Avenue, Lincoln Park, Ch...","41.928856, -87.663962","41.917776, -87.691433","Southport Ave & Wrightwood Ave, West Wrightwoo...",60613


In [599]:
df_lat_lng['zip_code'].value_counts()

60611    143
60614    100
60610     90
60601     82
60605     71
60617     69
60606     68
60607     68
60654     63
60622     62
60625     61
60613     61
60661     57
60612     56
60628     53
60657     52
60608     51
60637     50
60616     49
60620     49
60640     44
60619     43
60647     42
60609     42
60643     35
60604     34
60618     33
60615     33
60602     33
60626     31
60645     30
60624     28
60623     27
60632     26
60603     25
60629     25
60649     25
60634     24
60636     24
60651     24
60653     23
60644     23
60621     23
60641     22
60660     20
60642     19
60202     18
60201     18
60633     16
60655     16
s, IL     16
60630     13
60639     12
60659     12
60302     11
60827     10
 9020     10
60804     10
inois     10
60707      9
60208      9
46394      9
60696      8
60646      8
60652      7
60712      6
60638      6
60406      6
60803      6
60076      6
60091      6
90615      5
60402      5
60203      4
-4580      4
60409      4
60805      4

In [600]:
df_lat_lng['zip_code'].nunique()

83

Some texts don't seem to have the zip code
 
* s, IL     16
* inois     10
*  9020     10
* 46394      9
* 90615      5
* -4580      4
* -6427      2
* 46320      1

Some of them are still in the US, but not in Illinois.

Some of them don't have a zip code at all.

## Wrong zip codes

The zip codes in Chicago starts with 60-, so I used regular expressions to find any zip codes that don't start with 60.


In [601]:
# https://www.guru99.com/python-dictionary-append.html
# lists in a dictionary
wrong_dict = {'index' : [], 'start_lat_lng': [], 'end_lat_lng': [], 'address' : [], 'zip_code' : []}

# https://www.geeksforgeeks.org/python-regex-cheat-sheet/
# https://pythonexamples.org/python-regex-check-if-string-starts-with-specific-word/
for i in range(len(df_lat_lng)):
    # Find any zip code not starting with 60
    if re.search('^60', df_lat_lng['zip_code'][i]) == None:
#     if re.search('.60', df_lat_lng['full_address'][i]) == None: # wrong
        # Store their index, start_lat_lng, end_lat_lng, and full_address into err_dict
        wrong_dict['index'].append(df_lat_lng['index'][i])
        wrong_dict['start_lat_lng'].append(df_lat_lng['start_lat_lng'][i])
        wrong_dict['end_lat_lng'].append(df_lat_lng['end_lat_lng'][i])
        wrong_dict['address'].append(df_lat_lng['full_address'][i])
        wrong_dict['zip_code'].append(df_lat_lng['zip_code'][i])
wrong_dict

{'index': [13,
  45,
  86,
  144,
  366,
  372,
  489,
  538,
  870,
  1055,
  1195,
  1350,
  1417,
  1638,
  1785,
  2225,
  2277,
  3094,
  3384,
  3645,
  3815,
  5358,
  6430,
  6499,
  7019,
  8841,
  9266,
  10727,
  10740,
  14526,
  14624,
  16412,
  25073,
  30244,
  33365,
  35668,
  42943,
  55094,
  64571,
  65674,
  403518,
  540503,
  725676,
  943854,
  1181800,
  1187825,
  1457772,
  2271890,
  2853240,
  3248483,
  3569667,
  3750105,
  4379438,
  4387295,
  4527112,
  4902131,
  5090477],
 'start_lat_lng': ['41.969107, -87.674215',
  '41.67, -87.66',
  '41.96, -87.64',
  '41.69, -87.53',
  '41.963320, -87.679309',
  '41.909843, -87.705346',
  '41.83, -87.6',
  '41.86, -87.61',
  '41.967125, -87.679051',
  '41.956048, -87.680429',
  '41.851381, -87.618856',
  '41.883811, -87.623897',
  '41.852516, -87.626532',
  '41.793503, -87.615884',
  '41.860302, -87.625906',
  '41.684958, -87.645342',
  '41.892021, -87.689427',
  '41.853177, -87.631841',
  '41.69, -87.52',
  '41

In [605]:
# Convert err_dict into a DataFrame. easily accessible and looking better
df_wr_dict = pd.DataFrame(wrong_dict)
print(len(df_wr_dict))
df_wr_dict.head()

57


,index,start_lat_lng,end_lat_lng,address,zip_code
0,13,"41.969107, -87.674215","41.881688, -87.629418","4801-4809, North Ravenswood Avenue, Andersonvi...",9020
1,45,"41.67, -87.66","41.71, -87.57","West Pullman, Calumet Park, Calumet Township, ...",-6427
2,86,"41.96, -87.64","41.963922, -87.638156","West Montrose Harbor Drive, Uptown, Chicago, L...",inois
3,144,"41.69, -87.53","41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...",46394
4,366,"41.963320, -87.679309","41.939430, -87.711670","4449-4455, North Damen Avenue, West Ravenswood...",9020


In [609]:
df_lat_lng['zip_code'].unique()

array(['60614', '60613', '60649', '60610', '60611', '60651', '60606',
       '60603', ' 9020', '60617', '60622', '60626', '60619', '60608',
       '60612', '60642', '60640', '-6427', '60601', '60657', '60641',
       '60647', '60632', '60602', 'inois', '60654', '60645', '60660',
       '60637', '60605', '60625', '60616', '60618', '60201', '60202',
       '46394', '60607', '60646', '60628', '60208', '60615', '60623',
       '60661', '60604', '60302', '60639', '-4580', '60624', '60696',
       '60653', '60636', '60620', '60659', '60629', '60621', '60609',
       '60644', '60630', 's, IL', '60643', '60638', '60409', '60633',
       '60827', '60712', '60652', '46320', '60634', '60203', '90615',
       '60655', '60076', '60402', '60091', '60707', '60804', '60803',
       '60805', '60406', '60453', '60472', '60176', '60304'], dtype=object)

In [606]:
df_wr_dict['zip_code'].value_counts()

s, IL    16
 9020    10
inois    10
46394     9
90615     5
-4580     4
-6427     2
46320     1
Name: zip_code, dtype: int64

In [611]:
df_wr_dict.isnull().sum()

index            0
start_lat_lng    0
end_lat_lng      0
address          0
zip_code         0
dtype: int64

In [582]:
wr_dict_new_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
                   '12303 S Ashland Ave, Calumet Park, IL 60827, USA',
 'Uptown, Chicago, IL 60613, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
                   'West Town, Chicago, IL 60651, USA',
 '3155 S Lake Shore Dr, Chicago, IL 60616, United States',
 'Parking lot, Chicago, IL 60605, USA',
 '4705 N Damen Ave, Chicago, IL 60625, USA',
 '4108 N Lincoln Ave, Chicago, IL 60618, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 'Millennium Park, Chicago, IL 60601, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'MLK Jr Dr & 56th St, Chicago, IL 60612, United States',
 'Parking lot, Chicago, IL 60605, USA',
                   'Major Taylor Trail & 115th St, Chicago, IL 60643, USA',
                   'Smith Park, Chicago, IL 60612, USA',
 '2130 S Wentworth Ave, Chicago, IL 60616, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '10733 S Ave F, Chicago, IL 60617, USA',
 'Armour Square, Chicago, IL 60616, USA',
 'E 47th St, Chicago, IL 60653, USA',
                   'Smith Park, Chicago, IL 60612, USA',
 '72 W Monroe St, Chicago, IL 60603, United States',
 '5410 N Damen Ave, Chicago, IL 60625, USA',
 '3050 Moe Dr, Chicago, IL 60616, USA',
 'Baseball Junior/Softball, Chicago, IL 60615, USA',
 'Bronzeville, Chicago, IL 60653, USA',
 'State St & 95th St, Chicago, IL 60619, United States',
 '5074 S Calumet Ave, Chicago, IL 60612, USA',
 'Chicago, IL 60616, United States',
 'East Side, Chicago, IL 60617, USA',
 '100 W Randolph St 2nd fl, Chicago, IL 60601, United States',
                   '2910 W Ferdinand St, Chicago, IL 60612, USA',
 '1736 S Michigan Ave, Chicago, IL 60616, USA',
 '4802 N Ravenswood Ave, Chicago, IL 60612, USA',
                   'Parking lot, Chicago, IL 60605, USA',
 'Chicago, IL 60640, United States',
 'Dan Ryan Woods Trails, Chicago, IL 60620, USA',
 '2150 W Lawrence Ave, Chicago, IL 60625, USA',
 'Damen Ave & Leland Ave, Chicago, IL 60625, United States',
 'Chicago, IL 60605, United States',
 'McCormick Place, Chicago, IL 60616, United States',
 'Wells St & 19th St, Chicago, IL 60616, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 'Lincoln Ave & Sunnyside Ave, Chicago, IL 60625, United States',
 '1719 S Clinton St, Chicago, IL 60616, USA',
 'Museum Campus, Chicago, IL 60605, USA',
 'Parking lot, Chicago, IL 60616, USA',
 'Chicago, IL 60633, United States',
 '338 E 51st St, Chicago, IL 60615, USA',
 'S Wolf Lake Blvd, Chicago, IL 60633, United States',
 'Chicago, IL 60633, United States',
 '5420-5400 N Hoyne Ave, Chicago, IL 60625, USA',
 '1400-1414 Michigan Ave, Chicago, IL 60605, USA',
 'Chicago, IL 60633, USA']

In [620]:
df_wr_dict['new_address'] = wr_dict_new_add
# df_wr_dict = df_wr_dict.drop(columns = 'new_add')
df_wr_dict.head()

,index,start_lat_lng,end_lat_lng,address,zip_code,new_address
0,13,"41.969107, -87.674215","41.881688, -87.629418","4801-4809, North Ravenswood Avenue, Andersonvi...",9020,"Ravenswood Ave & Lawrence Ave, Chicago, IL 606..."
1,45,"41.67, -87.66","41.71, -87.57","West Pullman, Calumet Park, Calumet Township, ...",-6427,"12303 S Ashland Ave, Calumet Park, IL 60827, USA"
2,86,"41.96, -87.64","41.963922, -87.638156","West Montrose Harbor Drive, Uptown, Chicago, L...",inois,"Uptown, Chicago, IL 60613, USA"
3,144,"41.69, -87.53","41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...",46394,"11251-11249 S Ave B, Chicago, IL 60617, USA"
4,366,"41.963320, -87.679309","41.939430, -87.711670","4449-4455, North Damen Avenue, West Ravenswood...",9020,"2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA"


In [621]:
# Replace United States to USA
df_wr_dict['new_address'] = df_wr_dict['new_address'].replace('.United States', ' USA', regex=True)
df_wr_dict['new_address']

0     Ravenswood Ave & Lawrence Ave, Chicago, IL 606...
1      12303 S Ashland Ave, Calumet Park, IL 60827, USA
2                        Uptown, Chicago, IL 60613, USA
3           11251-11249 S Ave B, Chicago, IL 60617, USA
4     2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA
5                     West Town, Chicago, IL 60651, USA
6          3155 S Lake Shore Dr, Chicago, IL 60616, USA
7                   Parking lot, Chicago, IL 60605, USA
8              4705 N Damen Ave, Chicago, IL 60625, USA
9            4108 N Lincoln Ave, Chicago, IL 60618, USA
10              301 E Cermak Rd, Chicago, IL 60616, USA
11              Millennium Park, Chicago, IL 60601, USA
12               12 E Cermak Rd, Chicago, IL 60616, USA
13          MLK Jr Dr & 56th St, Chicago, IL 60612, USA
14                  Parking lot, Chicago, IL 60605, USA
15    Major Taylor Trail & 115th St, Chicago, IL 606...
16                   Smith Park, Chicago, IL 60612, USA
17         2130 S Wentworth Ave, Chicago, IL 606

In [622]:
df_wr_dict['zip_code'] = df_wr_dict['new_address'].str[-10:-5]
df_wr_dict['zip_code'].value_counts()

60616    12
60625     7
60612     6
60617     6
60605     6
60633     4
60615     2
60653     2
60640     2
60601     2
60827     1
60618     1
60643     1
60651     1
60603     1
60613     1
60619     1
60620     1
Name: zip_code, dtype: int64

In [623]:
df_wr_dict.head()

,index,start_lat_lng,end_lat_lng,address,zip_code,new_address
0,13,"41.969107, -87.674215","41.881688, -87.629418","4801-4809, North Ravenswood Avenue, Andersonvi...",60640,"Ravenswood Ave & Lawrence Ave, Chicago, IL 606..."
1,45,"41.67, -87.66","41.71, -87.57","West Pullman, Calumet Park, Calumet Township, ...",60827,"12303 S Ashland Ave, Calumet Park, IL 60827, USA"
2,86,"41.96, -87.64","41.963922, -87.638156","West Montrose Harbor Drive, Uptown, Chicago, L...",60613,"Uptown, Chicago, IL 60613, USA"
3,144,"41.69, -87.53","41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...",60617,"11251-11249 S Ave B, Chicago, IL 60617, USA"
4,366,"41.963320, -87.679309","41.939430, -87.711670","4449-4455, North Damen Avenue, West Ravenswood...",60625,"2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA"


If the coordinate doesn't have any zip code, then choose the nearest shop to get the zip code. 

In [ ]:
df_lat_lng # 2405
df_wr_dict # 57

In [624]:
df_wr_dict.isnull().sum()

index            0
start_lat_lng    0
end_lat_lng      0
address          0
zip_code         0
new_address      0
dtype: int64

In [625]:
print(df_lat_lng['zip_code'].nunique())
print(df_lat_lng['zip_code'].unique())

83
['60614' '60613' '60649' '60610' '60611' '60651' '60606' '60603' ' 9020'
 '60617' '60622' '60626' '60619' '60608' '60612' '60642' '60640' '-6427'
 '60601' '60657' '60641' '60647' '60632' '60602' 'inois' '60654' '60645'
 '60660' '60637' '60605' '60625' '60616' '60618' '60201' '60202' '46394'
 '60607' '60646' '60628' '60208' '60615' '60623' '60661' '60604' '60302'
 '60639' '-4580' '60624' '60696' '60653' '60636' '60620' '60659' '60629'
 '60621' '60609' '60644' '60630' 's, IL' '60643' '60638' '60409' '60633'
 '60827' '60712' '60652' '46320' '60634' '60203' '90615' '60655' '60076'
 '60402' '60091' '60707' '60804' '60803' '60805' '60406' '60453' '60472'
 '60176' '60304']


In [626]:
print(df_wr_dict['zip_code'].nunique())
print(df_wr_dict['zip_code'].unique())

18
['60640' '60827' '60613' '60617' '60625' '60651' '60616' '60605' '60618'
 '60601' '60612' '60643' '60653' '60603' '60615' '60619' '60620' '60633']


In [627]:
set(df_wr_dict['zip_code'].unique()).difference(df_lat_lng['zip_code'].unique())

set()

In [642]:
df_wr_dict['index'].unique()

array([     13,      45,      86,     144,     366,     372,     489,
           538,     870,    1055,    1195,    1350,    1417,    1638,
          1785,    2225,    2277,    3094,    3384,    3645,    3815,
          5358,    6430,    6499,    7019,    8841,    9266,   10727,
         10740,   14526,   14624,   16412,   25073,   30244,   33365,
         35668,   42943,   55094,   64571,   65674,  403518,  540503,
        725676,  943854, 1181800, 1187825, 1457772, 2271890, 2853240,
       3248483, 3569667, 3750105, 4379438, 4387295, 4527112, 4902131,
       5090477], dtype=int64)

In [644]:
df_wr_dict['zip_code'][df_wr_dict['index'] == 86]
# a['zip_code'][a['index'] == i]

2    60613
Name: zip_code, dtype: object

In [648]:
a = 0

for i in df_wr_dict['index']:
    df_lat_lng['zip_code'][df_lat_lng['index'] == i] = df_wr_dict['zip_code'][df_wr_dict['index'] == i][a]
    a += 1

C:\Users\GIS\AppData\Local\Temp/ipykernel_25284/327336763.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lat_lng['zip_code'][df_lat_lng['index'] == i] = df_wr_dict['zip_code'][df_wr_dict['index'] == i][a]


In [649]:
print(df_lat_lng['zip_code'].nunique())
print(df_lat_lng['zip_code'].unique())

75
['60614' '60613' '60649' '60610' '60611' '60651' '60606' '60603' '60640'
 '60617' '60622' '60626' '60619' '60608' '60612' '60642' '60827' '60601'
 '60657' '60641' '60647' '60632' '60602' '60654' '60645' '60660' '60637'
 '60605' '60625' '60616' '60618' '60201' '60202' '60607' '60646' '60628'
 '60208' '60615' '60623' '60661' '60604' '60302' '60639' '60624' '60696'
 '60653' '60636' '60620' '60659' '60629' '60621' '60609' '60644' '60630'
 '60643' '60638' '60409' '60633' '60712' '60652' '60634' '60203' '60655'
 '60076' '60402' '60091' '60707' '60804' '60803' '60805' '60406' '60453'
 '60472' '60176' '60304']


In [650]:
df_lat_lng.to_csv('df_lat_lng_code.csv')

In [193]:
# Copy the first row of df for the DataFrame structure
df_lat_lng1 = df.loc[:0]

for i in df_wr_dict['index']:
    df_lat_lng1 = df_lat_lng1.append(df[df.index == i])

# Exclude the first row
df_lat_lng1 = df_lat_lng1[1:]
print(len(df_lat_lng1))
df_lat_lng1.head()

50


,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
21,21,55D7218E9B555FE3,electric_bike,2020-10-26 10:29:35,2020-10-26 10:29:38,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894789,-87.730887,41.894764,-87.730910,casual,0,00:00:03,41.894,-87.730,"41.894, -87.730"
22,22,4B3EE1046889E2A3,electric_bike,2020-10-26 10:57:23,2020-10-26 10:57:24,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894768,-87.730913,41.894752,-87.730927,casual,0,00:00:01,41.894,-87.730,"41.894, -87.730"
36,36,131CF67E769A37B0,electric_bike,2020-10-13 16:31:42,2020-10-13 16:52:03,Ashland Ave & Division St,210.0,Pine Grove Ave & Irving Park Rd,254.0,41.903348,-87.667981,41.954377,-87.648298,casual,0,00:20:21,41.903,-87.667,"41.903, -87.667"
49,49,902A1458A729A722,electric_bike,2020-10-13 17:26:38,2020-10-13 17:44:00,NaN,NaN,St. Louis Ave & Fullerton Ave,503.0,41.930000,-87.730000,41.925195,-87.714581,casual,0,00:17:22,41.93,-87.73,"41.93, -87.73"
50,50,745E8688DA89AD00,electric_bike,2020-10-13 17:03:55,2020-10-13 17:08:30,Kosciuszko Park,499.0,NaN,NaN,41.930533,-87.723761,41.930000,-87.730000,casual,0,00:04:35,41.930,-87.723,"41.930, -87.723"


In [194]:
df_lat_lng1['zip_code'] = df_lat_lng['zip_code']
df_lat_lng1.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,zip_code
21,21,55D7218E9B555FE3,electric_bike,2020-10-26 10:29:35,2020-10-26 10:29:38,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894789,-87.730887,41.894764,-87.730910,casual,0,00:00:03,41.894,-87.730,"41.894, -87.730",46394
22,22,4B3EE1046889E2A3,electric_bike,2020-10-26 10:57:23,2020-10-26 10:57:24,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894768,-87.730913,41.894752,-87.730927,casual,0,00:00:01,41.894,-87.730,"41.894, -87.730",46394
36,36,131CF67E769A37B0,electric_bike,2020-10-13 16:31:42,2020-10-13 16:52:03,Ashland Ave & Division St,210.0,Pine Grove Ave & Irving Park Rd,254.0,41.903348,-87.667981,41.954377,-87.648298,casual,0,00:20:21,41.903,-87.667,"41.903, -87.667",46394
49,49,902A1458A729A722,electric_bike,2020-10-13 17:26:38,2020-10-13 17:44:00,NaN,NaN,St. Louis Ave & Fullerton Ave,503.0,41.930000,-87.730000,41.925195,-87.714581,casual,0,00:17:22,41.93,-87.73,"41.93, -87.73",46394
50,50,745E8688DA89AD00,electric_bike,2020-10-13 17:03:55,2020-10-13 17:08:30,Kosciuszko Park,499.0,NaN,NaN,41.930533,-87.723761,41.930000,-87.730000,casual,0,00:04:35,41.930,-87.723,"41.930, -87.723",46394


In [195]:
df_lat_lng1.isnull().sum()

Unnamed: 0             0
ride_id                0
rideable_type          0
started_at             0
ended_at               0
start_station_name    15
start_station_id      16
end_station_name      13
end_station_id        13
start_lat              0
start_lng              0
end_lat                0
end_lng                0
member_casual          0
riding_days            0
riding_time            0
start_lat_short        0
start_lng_short        0
lat_lng_short          0
zip_code               0
dtype: int64

In [140]:
[df_lat_lng1.index == 21]

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,zip_code
21,21,55D7218E9B555FE3,electric_bike,2020-10-26 10:29:35,2020-10-26 10:29:38,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894789,-87.730887,41.894764,-87.73091,casual,0,00:00:03,41.894,-87.730,"41.894, -87.730",46394


In [148]:
df_wrong_add = df_lat_lng1[:0]

for i in wrong_dict['index']:
    df_wrong_add = df_wrong_add.append(df_lat_lng1[df_lat_lng1.index == i])

df_wrong_add = df_wrong_add[0:]
len(df_wrong_add)
df_wrong_add

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short,zip_code
21,21,55D7218E9B555FE3,electric_bike,2020-10-26 10:29:35,2020-10-26 10:29:38,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894789,-87.730887,41.894764,-87.730910,casual,0,00:00:03,41.894,-87.730,"41.894, -87.730",46394
22,22,4B3EE1046889E2A3,electric_bike,2020-10-26 10:57:23,2020-10-26 10:57:24,WATSON TESTING - DIVVY,676.0,WATSON TESTING - DIVVY,676.0,41.894768,-87.730913,41.894752,-87.730927,casual,0,00:00:01,41.894,-87.730,"41.894, -87.730",46394
36,36,131CF67E769A37B0,electric_bike,2020-10-13 16:31:42,2020-10-13 16:52:03,Ashland Ave & Division St,210.0,Pine Grove Ave & Irving Park Rd,254.0,41.903348,-87.667981,41.954377,-87.648298,casual,0,00:20:21,41.903,-87.667,"41.903, -87.667",46394
49,49,902A1458A729A722,electric_bike,2020-10-13 17:26:38,2020-10-13 17:44:00,NaN,NaN,St. Louis Ave & Fullerton Ave,503.0,41.930000,-87.730000,41.925195,-87.714581,casual,0,00:17:22,41.93,-87.73,"41.93, -87.73",46394
50,50,745E8688DA89AD00,electric_bike,2020-10-13 17:03:55,2020-10-13 17:08:30,Kosciuszko Park,499.0,NaN,NaN,41.930533,-87.723761,41.930000,-87.730000,casual,0,00:04:35,41.930,-87.723,"41.930, -87.723",46394
76,76,0300CCEF9E951924,electric_bike,2020-10-23 17:30:39,2020-10-23 17:43:01,Franklin St & Jackson Blvd,36.0,Halsted St & Clybourn Ave,331.0,41.878082,-87.635060,41.909731,-87.648075,casual,0,00:12:22,41.878,-87.635,"41.878, -87.635",46320
77,77,33B6FE36FE6FC36A,electric_bike,2020-10-23 16:44:07,2020-10-23 16:49:40,NaN,NaN,NaN,NaN,41.910000,-87.650000,41.910000,-87.640000,casual,0,00:05:33,41.91,-87.65,"41.91, -87.65",46394
105,105,968C869BB88A44AC,electric_bike,2020-10-14 09:14:32,2020-10-14 09:26:09,NaN,NaN,NaN,NaN,41.780000,-87.590000,41.780000,-87.590000,casual,0,00:11:37,41.78,-87.59,"41.78, -87.59",46394
106,106,FDFB463A20EC30BA,electric_bike,2020-10-13 20:14:32,2020-10-13 20:41:53,State St & Randolph St,44.0,Michigan Ave & Lake St,52.0,41.884694,-87.627953,41.886004,-87.624520,casual,0,00:27:21,41.884,-87.627,"41.884, -87.627",46394
137,137,70285938CC62BF4A,electric_bike,2020-10-06 13:58:01,2020-10-06 14:22:51,Theater on the Lake,177.0,Wilton Ave & Belmont Ave,117.0,41.926219,-87.630876,41.940122,-87.653008,casual,0,00:24:50,41.926,-87.630,"41.926, -87.630",46394


In [162]:
for i in df_wr_dict['index']:
    print(geo_loc.geocode(df_wr_dict['lat_lng_short'][df_wr_dict['index'] == i]))
    break
    

None


In [174]:
df_wr_dict.head()

,index,lat_lng_short,address
0,21,"41.66, -87.52","129th Street, Hammond, Lake County, Indiana, 4..."
1,22,"41.66, -87.53","South State Line Road, Hegewisch, Chicago, Hyd..."
2,36,"41.67, -87.53","South State Line Road, Hegewisch, Chicago, Hyd..."
3,49,"41.68, -87.52","Illiana Marina Trail, Hammond, Lake County, In..."
4,50,"41.68, -87.53","Burnham Greenway Black Paved Trail, Hegewisch,..."


In [169]:
geo_loc.geocode('41.66, -87.52').address

'129th Street, Hammond, Lake County, Indiana, 46394, United States'

In [165]:
df_wr_dict['lat_lng_short'][df_wr_dict['index'] == i]

0    41.66, -87.52
Name: lat_lng_short, dtype: object

I manually searched every lat and len in df_err_dict. 

Because I think it was the best solution I could come up with at this moment...

In [122]:
crr_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
 '12303 S Ashland Ave, Calumet Park, IL 60827, United States',
 'Uptown, Chicago, IL 60613, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, United States',
 '1736 S Michigan Ave, Chicago, IL 60616, United States',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
 'Douglas, Chicago, IL 60616, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'Major Taylor Trail & 115th St, Chicago, IL 60643, United States',
 '10729 S Ave F, Chicago, IL 60617, USA',
 'Parking lot, Chicago, IL 60616, USA']

['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
 '12303 S Ashland Ave, Calumet Park, IL 60827, United States',
 'Uptown, Chicago, IL 60613, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, United States',
 '1736 S Michigan Ave, Chicago, IL 60616, United States',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
 'Douglas, Chicago, IL 60616, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'Major Taylor Trail & 115th St, Chicago, IL 60643, United States',
 '10729 S Ave F, Chicago, IL 60617, USA',
 'Parking lot, Chicago, IL 60616, USA']

In [85]:
# Add crr_add to df_err_dict
df_err_dict['crr_add'] = crr_add
df_err_dict

,index,lat_lng,text,crr_add
0,13,"41.969107, -87.674215","Ravenswood Ave & Lawrence Ave, North Ravenswoo...","Ravenswood Ave & Lawrence Ave, Chicago, IL 606..."
1,29,"41.67, -87.66","West Pullman, Calumet Park, Calumet Township, ...","12303 S Ashland Ave, Calumet Park, IL 60827, U..."
2,50,"41.96, -87.64","West Montrose Harbor Drive, Uptown, Chicago, L...","Uptown, Chicago, IL 60613, United States"
3,78,"41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...","11251-11249 S Ave B, Chicago, IL 60617, United..."
4,102,"41.857881, -87.624555","70, East 18th Street, South Loop, Near South S...","1736 S Michigan Ave, Chicago, IL 60616, United..."
5,132,"41.963320, -87.679309","Damen Ave & Sunnyside Ave, West Sunnyside Aven...","2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA"
6,146,"41.83, -87.6","Oakland, Chicago, Cook County, Illinois, Unite...","Douglas, Chicago, IL 60616, USA"
7,186,"41.851381, -87.618856","McCormick Place, South Doctor Martin Luther Ki...","301 E Cermak Rd, Chicago, IL 60616, USA"
8,193,"41.852516, -87.626532","Cermak-McCormick Place, East Cermak Road, Near...","12 E Cermak Rd, Chicago, IL 60616, USA"
9,209,"41.684958, -87.645342","Major Taylor Trail, Placerdale, West Pullman, ...","Major Taylor Trail & 115th St, Chicago, IL 606..."


(1) re.search('.60', df_err_dict['crr_add'][0]).span() says that its zip code position is (42, 45),
but (2) the actual position is (43, 48), so add 1 to start and 3 to end.

(1)
* re.search('.60', df_err_dict['crr_add'][0]).span()[0] --> output = (42)
* re.search('.60', df_err_dict['crr_add'][0]).span()[1] --> output = (45)

(2) the actual (43:48), start +1, end +3
* re.search('.60', df_err_dict['crr_add'][0]).span()[0]+1 --> output = (42+3) = (45)
* re.search('.60', df_err_dict['crr_add'][0]).span()[1]+3 --> output = (45+3) = (48)




In [123]:
# Check if the start and end position are right
print(df_err_dict['crr_add'][0][42:45])
print(df_err_dict['crr_add'][0][43:48]) # (42, 45), actual (43:48), start +1, end +3

 60
60640


In [116]:
zip_code = []

# Find the zip code position in the address
for i in range(len(df_err_dict['crr_add'])):
    if re.search('.60', df_err_dict['crr_add'][i]) != None:
        # Extract start and end position
        start = re.search('.60', df_err_dict['crr_add'][i]).span()[0]+1 # start
        end = re.search('.60', df_err_dict['crr_add'][i]).span()[1]+3 # end
        # Add to the list
        zip_code.append(df_err_dict['crr_add'][i][start:end])

zip_code

['60640',
 '60827',
 '60613',
 '60617',
 '60616',
 '60625',
 '60616',
 '60616',
 '60616',
 '60643',
 '60617',
 '60616']

In [121]:
df_err_dict['zip_code'] = zip_code
df_err_dict

,index,lat_lng,text,crr_add,zip_code
0,13,"41.969107, -87.674215","Ravenswood Ave & Lawrence Ave, North Ravenswoo...","Ravenswood Ave & Lawrence Ave, Chicago, IL 606...",60640
1,29,"41.67, -87.66","West Pullman, Calumet Park, Calumet Township, ...","12303 S Ashland Ave, Calumet Park, IL 60827, U...",60827
2,50,"41.96, -87.64","West Montrose Harbor Drive, Uptown, Chicago, L...","Uptown, Chicago, IL 60613, United States",60613
3,78,"41.69, -87.53","Eggers Grove, Burnham Greenway Blue Paved Trai...","11251-11249 S Ave B, Chicago, IL 60617, United...",60617
4,102,"41.857881, -87.624555","70, East 18th Street, South Loop, Near South S...","1736 S Michigan Ave, Chicago, IL 60616, United...",60616
5,132,"41.963320, -87.679309","Damen Ave & Sunnyside Ave, West Sunnyside Aven...","2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA",60625
6,146,"41.83, -87.6","Oakland, Chicago, Cook County, Illinois, Unite...","Douglas, Chicago, IL 60616, USA",60616
7,186,"41.851381, -87.618856","McCormick Place, South Doctor Martin Luther Ki...","301 E Cermak Rd, Chicago, IL 60616, USA",60616
8,193,"41.852516, -87.626532","Cermak-McCormick Place, East Cermak Road, Near...","12 E Cermak Rd, Chicago, IL 60616, USA",60616
9,209,"41.684958, -87.645342","Major Taylor Trail, Placerdale, West Pullman, ...","Major Taylor Trail & 115th St, Chicago, IL 606...",60643


In [259]:
# copy df1
df2 = df1[:]

In [234]:
import pdb

In [ ]:
df2['zip_code'].value_counts().sum()

In [336]:
# Change the wrong data in df2 to the correct data in df_err_dict
add = 0
for i in df_err_dict['index']:
#     pdb.set_trace()
    df2['zip_code'].loc[df2.index == i] = df_err_dict['zip_code'].loc[df_err_dict['index'] == i][add]
    add += 1

In [337]:
df2['zip_code'].value_counts().sum()

330

In [340]:
df2.isnull().sum()

ride_id                 0
rideable_type           0
started_at              0
ended_at                0
start_station_name     45
start_station_id       45
end_station_name      101
end_station_id        102
start_lat               0
start_lng               0
end_lat                 0
end_lng                 0
member_casual           0
riding_days             0
riding_time             0
start_lat_short         0
start_lng_short         0
lat_lng                 0
full_address            0
country                 0
zip_code                0
dtype: int64

In [341]:
df2.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng,full_address,country,zip_code
0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926100,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926100, -87.638977","Lakeview Ave & Fullerton Pkwy, North Lakeview ...",United States,60614
1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948172, -87.663911","CVS Pharmacy, 3637, North Southport Avenue, Wr...",United States,60613
2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585371,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773462, -87.585371","Stony Island Ave & 67th St, East 67th Street, ...",United States,60619
3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950854,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950854, -87.659244","Clark St & Grace St, West Grace Street, Pine G...",United States,60613
4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928856,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928856, -87.663962","Southport Ave & Wrightwood Ave, West Wrightwoo...",United States,60614


In [345]:
df2['lat_lng_short'] = df2['start_lat_short']+', '+df2['start_lng_short']
df2.head()

C:\Users\GIS\AppData\Local\Temp/ipykernel_25336/1011362197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['lat_lng_short'] = df2['start_lat_short']+', '+df2['start_lng_short']


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng,full_address,country,zip_code,lat_lng_short
0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926100,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926100, -87.638977","Lakeview Ave & Fullerton Pkwy, North Lakeview ...",United States,60614,"41.926, -87.638"
1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948172, -87.663911","CVS Pharmacy, 3637, North Southport Avenue, Wr...",United States,60613,"41.948, -87.663"
2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585371,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773462, -87.585371","Stony Island Ave & 67th St, East 67th Street, ...",United States,60619,"41.773, -87.585"
3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950854,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950854, -87.659244","Clark St & Grace St, West Grace Street, Pine G...",United States,60613,"41.950, -87.659"
4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928856,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928856, -87.663962","Southport Ave & Wrightwood Ave, West Wrightwoo...",United States,60614,"41.928, -87.663"


In [350]:
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,riding_days,riding_time,start_lat_short,start_lng_short,lat_lng_short
0,0,ACB6B40CF5B9044C,electric_bike,2020-10-31 19:39:43,2020-10-31 19:57:12,Lakeview Ave & Fullerton Pkwy,313.0,Rush St & Hubbard St,125.0,41.926101,-87.638977,41.890345,-87.626068,casual,0,00:17:29,41.926,-87.638,"41.926, -87.638"
1,1,DF450C72FD109C01,electric_bike,2020-10-31 23:50:08,2020-11-01 00:04:16,Southport Ave & Waveland Ave,227.0,Kedzie Ave & Milwaukee Ave,260.0,41.948172,-87.663911,41.929528,-87.707818,casual,0,00:14:08,41.948,-87.663,"41.948, -87.663"
2,2,B6396B54A15AC0DF,electric_bike,2020-10-31 23:00:01,2020-10-31 23:08:22,Stony Island Ave & 67th St,102.0,University Ave & 57th St,423.0,41.773462,-87.585372,41.791455,-87.600050,casual,0,00:08:21,41.773,-87.585,"41.773, -87.585"
3,3,44A4AEE261B9E854,electric_bike,2020-10-31 22:16:43,2020-10-31 22:19:35,Clark St & Grace St,165.0,Broadway & Sheridan Rd,256.0,41.950855,-87.659244,41.952809,-87.650103,casual,0,00:02:52,41.950,-87.659,"41.950, -87.659"
4,4,10B7DD76A6A2EB95,electric_bike,2020-10-31 19:38:19,2020-10-31 19:54:32,Southport Ave & Wrightwood Ave,190.0,Stave St & Armitage Ave,185.0,41.928857,-87.663962,41.917777,-87.691434,casual,0,00:16:13,41.928,-87.663,"41.928, -87.663"


In [363]:
a = pd.DataFrame(df2.groupby(['zip_code','lat_lng_short'])['zip_code'].count())
a = a.rename(columns = {'zip_code': 'count'}).reset_index()
a

,zip_code,lat_lng_short,count
0,60201,"42.023, -87.699",1
1,60201,"42.039, -87.699",1
2,60201,"42.047, -87.683",1
3,60201,"42.048, -87.698",1
4,60201,"42.05, -87.68",1
...,...,...,...
325,60661,"41.888, -87.644",1
326,60827,"41.656, -87.598",1
327,60827,"41.657, -87.610",1
328,60827,"41.658, -87.610",1


In [365]:
a['zip_code'].nunique()

48

In [372]:
df['lat_lng_short'].nunique()

2405

In [377]:
for i in range 
df2['lat_lng_short'] in df['lat_lng_short']

True

In [379]:
df_set = set(df['lat_lng_short'])
df2_set = set(set(df2['lat_lng_short']))

len(df2_set & df_set) # set intersection

330

In [381]:
# prints the missing and additional elements in list1
# print("Missing values in first list:", (set(df2_set).difference(df_set)))
print("Additional values in first list:", (set(df_set).difference(df2_set)))

Additional values in first list: {'41.780, -87.615', '41.948, -87.740', '41.89, -87.6', '41.906, -87.634', '41.958, -87.649', '41.883, -87.626', '41.884, -87.638', '41.872, -87.746', '41.84, -87.76', '41.956, -87.669', '41.780, -87.591', '41.7, -87.71', '41.895, -87.634', '41.939, -87.645', '41.878, -87.649', '41.64, -87.62', '41.895, -87.627', '41.814, -87.597', '41.7, -87.72', '41.867, -87.622', '41.799, -87.587', '41.97, -87.71', '41.854, -87.663', '41.84, -87.77', '41.896, -87.631', '41.85, -87.61', '41.68, -87.57', '41.69, -87.54', '41.858, -87.656', '41.853, -87.620', '41.94, -87.8', '41.877, -87.634', '41.895, -87.624', '41.879, -87.638', '41.887, -87.620', '41.896, -87.621', '41.82, -87.66', '41.897, -87.622', '41.89, -87.65', '41.881, -87.639', '41.904, -87.631', '41.929, -87.642', '41.81, -87.64', '41.7, -87.54', '41.874, -87.627', '41.94, -87.74', '42.01, -87.7', '41.912, -87.634', '41.880, -87.617', '41.87, -87.69', '41.92, -87.71', '41.893, -87.639', '41.903, -87.683', '41

In [47]:
print(len(df), len(df_lat_lng))

5136261 5136261


In [167]:
geo_loc.geocode(df_lat_lng['lat_lng_short'][0]).address

'14049, South Burnham Avenue, Burnham, Thornton Township, Cook County, Illinois, 60633, United States'

In [75]:
# zip code
geo_loc.geocode(df['start_lat'][1].astype(str)[:10]+","+df['start_lng'][1].astype(str)[:11]).address[-20:-15]
# USA
geo_loc.geocode(df['start_lat'][1].astype(str)[:10]+","+df['start_lng'][1].astype(str)[:11]).address[-13:]

'60613'

In [151]:
geo_loc = Nominatim(user_agent="GetLoc")

In [39]:
loc_list = []

# Get location with geocode
# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")
geo_loc.geocode(Latitude+","+Longitude)
# geo_loc.geocode(df['start_lat'][1].astype(str)[:10]+","+df['start_lng'][1].astype(str)[:11])


Location(CVS Pharmacy, 3637, North Southport Avenue, Wrigleyville, Lake View, Chicago, Lake View Township, Cook County, Illinois, 60613, United States, (41.948237, -87.66368319241383, 0.0))

In [98]:
geo_loc.geocode('41.87, -87.63')

Location(960-996, South Park Terrace, Dearborn Park, Loop, Chicago, Cook County, Illinois, 60605, United States, (41.870180399999995, -87.63028242517899, 0.0))

In [ ]:
loc_list = []
# 1913 - 
for i in range(len(df_lat_lng)):
#     zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address[-20:-15]
    zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address
    loc_list.append(zip_code)

In [99]:
loc_list = []
# 1913 - 
for i in range(len(df_lat_lng)):
#     zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address[-20:-15]
    zip_code = geo_loc.geocode(df_lat_lng['lat_lng'][i]).address
    loc_list.append(zip_code)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=41.830115%2C+-87.670379&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

1412

In [ ]:
zip_code = []

# Find the zip code position in the address
for i in range(len(df_err_dict['crr_add'])):
    if re.search('.60', df_err_dict['crr_add'][i]) != None:
        
        # Find the zip code position in the address
#         print(re.search('.60', df_err_dict['crr_add'][i]).span())
        
        # Extract start and end position
        start = re.search('.60', df_err_dict['crr_add'][i]).span()[0]+1 # start
        end = re.search('.60', df_err_dict['crr_add'][i]).span()[1]+3 # end
        
        zip_code.append(df_err_dict['crr_add'][i][start:end])

zip_code

In [ ]:
crr_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
 '12303 S Ashland Ave, Calumet Park, IL 60827, United States',
 'Uptown, Chicago, IL 60613, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, United States',
 '1736 S Michigan Ave, Chicago, IL 60616, United States',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
 'Douglas, Chicago, IL 60616, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'Major Taylor Trail & 115th St, Chicago, IL 60643, United States',
 '10729 S Ave F, Chicago, IL 60617, USA',
 'Parking lot, Chicago, IL 60616, USA']

In [ ]:
df_lat_lng1 = df_lat_lng[:802]
df_lat_lng2 = df_lat_lng[802:1605]
df_lat_lng3 = df_lat_lng[1605:]

print(len(df_lat_lng1),
      len(df_lat_lng2),
      len(df_lat_lng3)
     )

# calling the nominatim tool
geo_loc = Nominatim(user_agent="GetLoc")

loc_list1 = []
loc_list2 = []
loc_list3 = []

# As df_lat_lng1 start from 0, I don't need to set the range specifically
for i in range(len(df_lat_lng1)):
    zip_code = geo_loc.geocode(df_lat_lng1['lat_lng_short'][i]).address
    loc_list1.append(zip_code)
print('df_lat_lng1 - Done')

# Set the range as df_lat_lng2 starts from 802 and ends at 1604
for j in range(df_lat_lng2['lat_lng_short'].index.start, df_lat_lng2['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng2['lat_lng_short'][j]).address
    loc_list2.append(zip_code)
print('df_lat_lng2 - Done')

# Set the range as df_lat_lng3 starts from 1605 and ends at 2404
for k in range(df_lat_lng3['lat_lng_short'].index.start, df_lat_lng3['lat_lng_short'].index.stop):
    zip_code = geo_loc.geocode(df_lat_lng3['lat_lng_short'][k]).address
    loc_list3.append(zip_code)
print('df_lat_lng3 - Done')


In [ ]:
wr_dict_new_add = ['Ravenswood Ave & Lawrence Ave, Chicago, IL 60640, United States',
                   '12303 S Ashland Ave, Calumet Park, IL 60827, USA',
 'Uptown, Chicago, IL 60613, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '2000-2018 W Sunnyside Ave, Chicago, IL 60625, USA',
                   'West Town, Chicago, IL 60651, USA',
 '3155 S Lake Shore Dr, Chicago, IL 60616, United States',
 'Parking lot, Chicago, IL 60605, USA',
 '4705 N Damen Ave, Chicago, IL 60625, USA',
 '4108 N Lincoln Ave, Chicago, IL 60618, USA',
 '301 E Cermak Rd, Chicago, IL 60616, USA',
 'Millennium Park, Chicago, IL 60601, USA',
 '12 E Cermak Rd, Chicago, IL 60616, USA',
 'MLK Jr Dr & 56th St, Chicago, IL 60612, United States',
 'Parking lot, Chicago, IL 60605, USA',
                   'Major Taylor Trail & 115th St, Chicago, IL 60643, USA',
                   'Smith Park, Chicago, IL 60612, USA',
 '2130 S Wentworth Ave, Chicago, IL 60616, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '10733 S Ave F, Chicago, IL 60617, USA',
 'Armour Square, Chicago, IL 60616, USA',
 'E 47th St, Chicago, IL 60653, USA',
                   'Smith Park, Chicago, IL 60612, USA',
 '72 W Monroe St, Chicago, IL 60603, United States',
 '5410 N Damen Ave, Chicago, IL 60625, USA',
 '3050 Moe Dr, Chicago, IL 60616, USA',
 'Baseball Junior/Softball, Chicago, IL 60615, USA',
 'Bronzeville, Chicago, IL 60653, USA',
 'State St & 95th St, Chicago, IL 60619, United States',
 '5074 S Calumet Ave, Chicago, IL 60612, USA',
 'Chicago, IL 60616, United States',
 'East Side, Chicago, IL 60617, USA',
 '100 W Randolph St 2nd fl, Chicago, IL 60601, United States',
                   '2910 W Ferdinand St, Chicago, IL 60612, USA',
 '1736 S Michigan Ave, Chicago, IL 60616, USA',
 '4802 N Ravenswood Ave, Chicago, IL 60612, USA',
                   'Parking lot, Chicago, IL 60605, USA',
 'Chicago, IL 60640, United States',
 'Dan Ryan Woods Trails, Chicago, IL 60620, USA',
 '2150 W Lawrence Ave, Chicago, IL 60625, USA',
 'Damen Ave & Leland Ave, Chicago, IL 60625, United States',
 'Chicago, IL 60605, United States',
 'McCormick Place, Chicago, IL 60616, United States',
 'Wells St & 19th St, Chicago, IL 60616, United States',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 '11251-11249 S Ave B, Chicago, IL 60617, USA',
 'Lincoln Ave & Sunnyside Ave, Chicago, IL 60625, United States',
 '1719 S Clinton St, Chicago, IL 60616, USA',
 'Museum Campus, Chicago, IL 60605, USA',
 'Parking lot, Chicago, IL 60616, USA',
 'Chicago, IL 60633, United States',
 '338 E 51st St, Chicago, IL 60615, USA',
 'S Wolf Lake Blvd, Chicago, IL 60633, United States',
 'Chicago, IL 60633, United States',
 '5420-5400 N Hoyne Ave, Chicago, IL 60625, USA',
 '1400-1414 Michigan Ave, Chicago, IL 60605, USA',
 'Chicago, IL 60633, USA']